In [1]:
import sys
sys.path.append('..')
from datasets import load_dataset
import os 
from utils import resize_preserve_aspect_ratio, pad_to_fixed_size
from PIL import Image, ImageOps
import numpy as np

workers = os.cpu_count()

dataset_id = "jordandavis/fashion_people_detections"
ds = load_dataset(dataset_id, split='train', num_proc=workers)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

In [4]:
def resize_image_pil(image_pil, max_size=1024):
    # Ensure image is in RGB
    if image_pil.mode != "RGB":
        image_pil = image_pil.convert("RGB")

    # Calculate new dimensions preserving aspect ratio
    width, height = image_pil.size
    scale = min(max_size / width, max_size / height)
    new_width = int(width * scale)
    new_height = int(height * scale)
    image_pil = image_pil.resize((new_width, new_height), Image.LANCZOS)

    # Calculate padding needed to reach 1024x1024
    pad_width = (max_size - new_width) // 2
    pad_height = (max_size - new_height) // 2

    # Apply padding symmetrically
    image_pil = ImageOps.expand(
        image_pil,
        border=(
            pad_width,
            pad_height,
            max_size - new_width - pad_width,
            max_size - new_height - pad_height,
        ),
        fill=(0, 0, 0),
    )

    return image_pil


def resize_images(batch):
    batch["image"] = resize_image_pil(batch["image"])
    return batch

In [ ]:
workers = os.cpu_count()
ds = ds.map(resize_images)

In [ ]:
from utils import convert_coco_polygons_to_mask, overlay_mask
from PIL import Image


def sanity_check(ds, row=102, mask_row=2):
    image = ds[row]["image"]
    width, height = image.size
    polygons = ds[row]["mask_metadata"][mask_row]["polygons"]
    label = ds[row]["mask_metadata"][mask_row]["label"]
    mask = convert_coco_polygons_to_mask(polygons, width, height)
    mask_image = Image.fromarray(mask)
    overlay = overlay_mask(image, mask_image, opacity=0.8)

    print(label)
    display(overlay)


row = 3
mask_row = 0
sanity_check(ds, row, mask_row)

### IF THE MASKS ARE ACCURATE THEN PUSH IT TO THE HUB ###

(800, 800)

In [ ]:
# updated_ds.push_to_hub(new_repo_id, commit_message="resized images")

### NEXT python yolo_dataset.py in the CLI ###

### THEN TRAIN THE YOLO MODEL in yolo_train.ipynb notebook ###

ds[0]['imaage']

In [6]:
resize_image_pil(ds[0]["image"]).size

(1024, 1024)